In [ ]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import sys
%matplotlib inline
import scipy.stats as st
import pprint
# Import API key
with open('/Users/tmlun/Downloads/api_keys.py', 'r') as g_key:
    mykey=g_key.read()
    mykey=mykey.split('=')
    mykey=mykey[1].strip()   
g_key = mykey.strip('"')
#from api_keys import g_key
gmaps.configure(api_key=g_key)

# Data File path (CSV)
#data = "./output_data/cities.csv"

In [ ]:
# Study data files
RMS_path = "./RMS.csv"
# Read the data
Crime_df = pd.read_csv(RMS_path)
print(Crime_df)


In [ ]:
#Clean the Zipcode Data
Crime_df = Crime_df.dropna(subset=['ZIP'])
print(Crime_df)

In [ ]:
Crime_df=Crime_df.loc[(Crime_df["ZIP"]!="0") & (Crime_df["ZIP"]!="1") & (Crime_df["ZIP"]!="78") & 
                                (Crime_df["ZIP"]!="11111") & (Crime_df["ZIP"]!="6278") & (Crime_df["ZIP"]!="50701")
                               & (Crime_df["ZIP"]!="81507")]
print(Crime_df)

In [ ]:
# of zipcodes
Zipcodes_df = Crime_df.loc[:,["ZIP"]]
print(Zipcodes_df)
print(type(Zipcodes_df))
Zipcodes_df["ZIP"].unique()

In [ ]:
Num_of_Crimes_grp = Crime_df.groupby("ZIP")
Num_of_Crimes = Num_of_Crimes_grp["INCIDENT_NUM"].count()
Num_of_Crimes_2 = Num_of_Crimes.astype(float)
Num_of_Crimes_3 = pd.DataFrame(Num_of_Crimes_2)
print(type(Num_of_Crimes_3))
print(Num_of_Crimes_2)
print(Num_of_Crimes_3)

In [ ]:
Zipcodes_df 

Zipcodes_df["Lat"] = ""
Zipcodes_df["Lon"] = ""

# Target zipcode
# Zipcodes_df_2["Zip"] = Zipcodes_df_2["ZIP"].astype(str)

print(Zipcodes_df.dtypes)
print(Zipcodes_df)

In [ ]:
# params = {"query": Zipcode,"key": g_key}

for index,row in Zipcodes_df.iterrows():

    # get zipcode info from df
    Zipcode = row['ZIP']
    
    # add keyword to params dict
#     params['query'] = Zipcode
#     params['key'] = g_key
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {Zipcode}.")
    # Build the endpoint URL
    target_url = ("https://maps.googleapis.com/maps/api/place/textsearch/json?query="+str(Zipcode)+"&key="+g_key)
    response = requests.get(target_url).json()
#     pprint.pprint(response)
    
    # extract results
    results = response['results']
    
    
    try:
        print(f"The lat is: {results[0]['geometry']['location']['lat']}")
        print(f"The lng is: {results[0]['geometry']['location']['lng']}")
        
        Zipcodes_df.loc[index, 'Lat'] = response['results'][0]['geometry']['location']['lat']
        Zipcodes_df.loc[index, 'Lon'] = response['results'][0]['geometry']['location']['lng']
        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
#     print("------------")

Zipcodes_df

In [ ]:
Zipcodes_df_2["Lat"] = pd.to_numeric(Zipcodes_df_2["Lat"])
Zipcodes_df_2["Lon"] = pd.to_numeric(Zipcodes_df_2["Lon"])


locations = Zipcodes_df_2[["Lat", "Lon"]]
Crime = Num_of_Crimes_3
print(len(locations))
print(len(Crime))

In [ ]:
fig= gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Crime, dissipating=False,
                                point_radius=5)

fig.add_layer(heat_layer)

fig